In [1]:
import swmmio
import pyswmm
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental import preprocessing


# desired_width = 500
# pd.set_option("display.width", desired_width)
# np.set_printoptions(linewidth=desired_width)
# pd.set_option("display.max_columns", 30)
import pprint
pd.set_option('display.max_columns', None)
# Ustawienie maksymalnej ilości wyświetlanych wierszy
pd.set_option('display.max_rows', None)


In [2]:
classes = pd.DataFrame(
    data={
        "classes": [
            "pump",
            "tank",
            "seepage_boxes",
            "diameter_increase",
            "diameter_reduction",
            "slope_increase",
            "slope_reduction",
            "depth_increase",
            "depth_reduction",
            "valid",
        ]
    }
)
classes

,classes
0,pump
1,tank
2,seepage_boxes
3,diameter_increase
4,diameter_reduction
5,slope_increase
6,slope_reduction
7,depth_increase
8,depth_reduction
9,valid


In [3]:
import sys
print(sys.path)

['c:\\Users\\Dell\\Documents\\Git\\stormwater-analysis\\stormwater_analysis\\data\\recomendations\\dataset', 'C:\\Users\\Dell\\AppData\\Local\\Programs\\Python\\Python310\\python310.zip', 'C:\\Users\\Dell\\AppData\\Local\\Programs\\Python\\Python310\\DLLs', 'C:\\Users\\Dell\\AppData\\Local\\Programs\\Python\\Python310\\lib', 'C:\\Users\\Dell\\AppData\\Local\\Programs\\Python\\Python310', 'c:\\Users\\Dell\\Documents\\Git\\stormwater-analysis\\venv', '', 'c:\\Users\\Dell\\Documents\\Git\\stormwater-analysis\\venv\\lib\\site-packages', 'c:\\Users\\Dell\\Documents\\Git\\stormwater-analysis\\venv\\lib\\site-packages\\win32', 'c:\\Users\\Dell\\Documents\\Git\\stormwater-analysis\\venv\\lib\\site-packages\\win32\\lib', 'c:\\Users\\Dell\\Documents\\Git\\stormwater-analysis\\venv\\lib\\site-packages\\Pythonwin']


In [4]:
import os
os.chdir('C:\\Users\\Dell\\Documents\\Git\\stormwater-analysis')

from stormwater_analysis.inp_manage.inp import SwmmModel

c:\Users\Dell\Documents\Git\stormwater-analysis\venv\lib\site-packages\keras\saving\saved_model\load.py:107: RuntimeWarning: Unexpected end-group tag: Not all data was converted
  metadata.ParseFromString(file_content)


In [7]:
INP_FILE = R"C:\Users\Dell\Documents\Git\stormwater-analysis\stormwater_analysis\data\recomendations\dataset\tank\01_tank.inp"

from stormwater_analysis.inp_manage.inp import SwmmModel
from stormwater_analysis.data.feature_engineering import feature_engineering

model = swmmio.Model(INP_FILE_01, include_rpt=True)

with pyswmm.Simulation(model.inp.path) as sim:
    for _ in sim:
        pass

conduits_data, nodes_data, subcatchments_data = feature_engineering(model)
swmm_model = SwmmModel(model, conduits_data, nodes_data, subcatchments_data)

AttributeError: '_UserObject' object has no attribute 'predict'